In [1]:
#Importando as bibliotecas
import pandas as pd
import os
import datetime as dt
import numpy as np
import regex as re
import numpy as np
from dateutil.relativedelta import relativedelta

In [2]:
os.chdir("C:/Users/User/OneDrive/Documentos/Kinea")

In [3]:
#Leitura dos arquivos
colunas = ["Cod.","ID Salesforce","Imóvel","Locação","Valor_1_ano"]
excel_antigo = pd.read_excel("Projeções Novas Locações_Anterior.xlsx",engine='openpyxl')
excel_novo = pd.read_excel("Projeções Novas Locações_Atual.xlsx",engine='openpyxl')
excel_antigo = excel_antigo.rename(columns = {excel_antigo.columns[10]:"Valor_1_ano_antigo"},inplace=False)
excel_novo = excel_novo.rename(columns = {excel_novo.columns[10]:"Valor_1_ano"},inplace=False)
excel_novo["Valor_1_ano"] = excel_novo["Valor_1_ano"]*excel_novo["Área "] #Calculando o valor total do pagamento do imóvel
excel_novo = excel_novo[colunas]
excel_antigo["Valor_1_ano_antigo"] = excel_antigo["Valor_1_ano_antigo"]*excel_antigo["Área "] #Calculando o valor total do pagamento do imóvel
excel_antigo = excel_antigo[["Cod.","ID Salesforce","Imóvel","Locação","Valor_1_ano_antigo"]]

In [4]:
#Juntando os dois arquivos
excel_junto = pd.merge(excel_novo,excel_antigo,how = "left",copy=True,left_on = ["Cod.","ID Salesforce","Imóvel"],right_on = ["Cod.","ID Salesforce","Imóvel"])
excel_junto = excel_junto.rename(columns = {"Locação_y":"Locação_antigo","Locação_x":"Locação_novo"})
z = []
for i in range(excel_junto.shape[0]):
    if all(pd.isna(excel_junto.iloc[i,])):
        z.append(i)
excel_junto = excel_junto.drop(z) #Retirando todas as linhas que tenham somente NAs
excel_junto["diferenca_data"] = excel_junto["Locação_novo"]-excel_junto["Locação_antigo"] #Diferença de datas de locação

In [5]:
#Sequência de datas para o arquivo separado
min_antigo = min(excel_junto["Locação_antigo"])
min_novo = min(excel_junto["Locação_novo"])
max_antigo = max(excel_junto["Locação_antigo"])
max_novo = max(excel_junto["Locação_novo"])
max_tot = max([max_antigo,max_novo])
min_tot = min([min_antigo,min_novo])
datas = []
current = min_tot
while current <= max_tot:
    datas.append(current)
    current += relativedelta(months = 1)
for i in range(len(datas)):
    datas[i] = datas[i].strftime("%m/%Y")

In [6]:
#Formatando as datas para %m/%Y
array = list(excel_junto["Locação_novo"].apply(str))
array1 = list(excel_junto["Locação_antigo"].apply(str))
for i in range(len(array)):
    array[i] = dt.datetime.strptime(array[i],"%Y-%m-%d %H:%M:%S")
    array[i] = array[i].strftime("%m/%Y")
    if not array1[i] == "NaT":
        array1[i] = dt.datetime.strptime(array1[i],"%Y-%m-%d %H:%M:%S")
        array1[i] = array1[i].strftime("%m/%Y")
excel_junto["Locação_novo"] = array
excel_junto["Locação_antigo"] = array1

In [7]:
#Função para retornar a classificação entre diferenças de datas
def diferenca(time):
    time = list(time)
    na = pd.isna(time)
    v = []
    for i in range(len(time)):
        if na[i]:
            v.append("Novo")
        else:
            dife = time[i]/dt.timedelta(days = 30)
            if dife < 0:
                v.append("Adiantou")
            elif dife > 0:
                v.append("Postergou")
            elif dife == 0:
                v.append("Igual")
    return(v)

In [8]:
excel_junto["diferenca_data"] = diferenca(excel_junto["diferenca_data"]) #Classificação de diferenças de datas
excel_junto["diferenca_valor"] = excel_junto["Valor_1_ano"]-excel_junto["Valor_1_ano_antigo"] #Diferença de valores

In [9]:
#Separando os dados
excel_separado = excel_junto.pivot(index = ["Cod.","ID Salesforce","Imóvel","diferenca_data","Locação_novo","Valor_1_ano","diferenca_valor"],columns = "Locação_antigo",values = "Valor_1_ano_antigo").reset_index()
excel_separado.drop(columns = ["NaT"])
excel_separado = excel_separado.pivot(index = ["Cod.","ID Salesforce","Imóvel","diferenca_data","diferenca_valor"],columns = "Locação_novo",values="Valor_1_ano").reset_index()

In [10]:
#Criando colunas para todos as datas entre a menor e a maior data de locação
excel_separado1 = excel_separado.iloc[0:,0:5]
nomes = excel_separado.columns[3:]
for i in datas:
    if i in nomes:
        excel_separado1 = pd.concat([excel_separado1,excel_separado[i]],axis = 1)
    else:
        excel_separado1[i] = np.NaN

In [11]:
#Inserindo os valores antigos e atuais nas colunas
for i in range(excel_separado1.shape[0]):
    cod = excel_separado1["Cod."][i]
    if pd.isna(cod):
        df = excel_junto[pd.isna(excel_junto["Cod."])]
    else:
        df = excel_junto[excel_junto["Cod."] == cod]
    diferenca = df["diferenca_data"]
    if not (diferenca.values in ["Igual","Novo"]):
        loc_antigo = np.where([j == df["Locação_antigo"].values[0] for j in excel_separado1.columns]) #Índice da coluna de valores antigos
        val_antigo = df["Valor_1_ano_antigo"].values #Valores antigos
        loc_novo = np.where([j == df["Locação_novo"].values[0] for j in excel_separado1.columns]) #Índice da coluna de valores atuais
        loc_antigo = loc_antigo[0][0]
        loc_novo = loc_novo[0][0]
        val_novo = df["Valor_1_ano"].values #Valores novos
        excel_separado1.iloc[i,loc_antigo] = val_antigo[0]
        excel_separado1.iloc[i,loc_novo] = val_novo[0]